In [1]:
import requests
import pandas as pd
import datetime

In [48]:
#returns a list of dicts which are meant to be transformed to a dataframe
def get_race_results(season,round):
    race_url= 'http://ergast.com/api/f1/'+season+'/'+round+'/results.json'
    r = requests.get(race_url)
    json=r.json()
    MRData=json["MRData"]
    RaceTable = MRData["RaceTable"]
    try:
        race = RaceTable["Races"][0]
    except:
        return []

    #Round Details
    round_season    = race["season"]
    round_round     = race["round"]
    round_circuit   = race["Circuit"]["circuitName"]
    round_date      = race["date"]

    #Results
    results = race["Results"]

    result_list = []
    for q_result in range(len(results)):
        #Driver Details
        driver = results[q_result]["Driver"]
        driver_name = driver["givenName"] + " " + driver["familyName"]
        driver_nationality = driver["nationality"]
        driver_code = driver["code"]
        #Constructor Details
        constructor = results[q_result]["Constructor"]
        constructor_name = constructor["name"]

        #Race Details
        race_position = results[q_result]["position"]
        race_points = results[q_result]["points"]
        race_laps = results[q_result]["laps"]
        try:
            race_time = results[q_result]["Time"]["millis"]
        except:
            race_time = None
        try:
            race_fastest_lap_rank = results[q_result]["FastestLap"]["rank"]
            race_fastest_lap_time = results[q_result]["FastestLap"]["Time"]["time"]
            race_fastest_lap_AverageSpeed = results[q_result]["FastestLap"]["AverageSpeed"]["speed"]
        except:
            race_fastest_lap_rank = None
            race_fastest_lap_time = None
            race_fastest_lap_AverageSpeed = None


        position = results[q_result]["position"]
        result_dict = {"season": round_season,
                        "round": round_round,
                        "circuit": round_circuit,
                        "date": round_date,
                        "driver_name": driver_name,
                        "driver_code": driver_code,
                        "driver_nationality": driver_nationality,
                        "constructor_name": constructor_name,
                        "race_position": race_position,
                        "race_points": race_points,
                        "race_laps": race_laps,
                        "race_time": race_time,
                        "race_fastest_lap_rank": race_fastest_lap_rank,
                        "race_fastest_lap_time": race_fastest_lap_time,
                        "race_fastest_lap_AverageSpeed":race_fastest_lap_AverageSpeed
                        }
        result_list.append(result_dict)
    return result_list

In [ ]:
first_year = 2005
last_year = 2023
race_list=[]

for season in range(first_year,last_year+1):
    season=str(season)
    #get num races
    race_url= 'http://ergast.com/api/f1/'+season+'/1/qualifying.json'
    r = requests.get(race_url)
    num_races=r.json()["MRData"]["total"]
    print("getting data for season "+season+ ". There are : "+num_races+ " number of races.")

    for round in range(int(num_races)):
        round=str(round+1)
        #print("getting data for season "+season+ " and round: "+round)
        race_list=race_list+get_race_results(season=season,round=round)
print("done")

In [59]:
#make list into df
df = pd.DataFrame(race_list)
df["ReadInTime"]=datetime.datetime.now()
print(df.shape)

(1854, 16)


In [60]:
#read df into sql server database
database="F1"
from sqlalchemy import create_engine
engine = create_engine('mssql+pyodbc://localhost/'+database+'?driver=SQL+Server+Native+Client+11.0')
df.to_sql('stg_race', engine, if_exists="append", index=False)

-1